### Importando Bibliotecas

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
import time

#### Setagem de Configuração de Navegação

In [18]:
# Configs
URL = 'https://monitoramento.semas.pa.gov.br/monitoramento/#/sig'
chrome_options = Options()
chrome_options.add_argument('ignore-certificate-errors')
# chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--ignore-certificate-errors')

# Instanciando Driver
chromedriver_path = chromedriver_autoinstaller.install()


### Acessando o Site e Instanciando Informações PMFS 

In [55]:
# Direcionamento para o site
driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)
driver.get(URL)

time.sleep(3) ## Delay de carregamento do Mapa

# Instancia dos dados no Mapa
agenda_verde = driver.find_element("xpath", '//*[@id="sidebar"]/div[1]/ul/li[1]')
agenda_verde.click()
time.sleep(1)

habilitar_pmfs = driver.find_element("xpath", '/html/body/div/div/mapa-sig/div[1]/div[2]/div[1]/uib-accordion/div/div[1]/div[2]/div/ul/li[2]/span/i[2]')
habilitar_pmfs.click()

time.sleep(2)

for i in range(10, 40):
    try:
        markers_infos = driver.find_element('css selector', '#sig > mapa-sig > div.mapa.sidebar-map.leaflet-container.leaflet-touch.leaflet-fade-anim.leaflet-grab.leaflet-touch-drag.leaflet-touch-zoom > div.leaflet-pane.leaflet-map-pane > div.leaflet-pane.leaflet-marker-pane > div:nth-child('+ str(i) + ') > i')
        markers_infos.click()
        time.sleep(2)
    except:
        pass

/tmp/ipykernel_64593/3890997894.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(options=chrome_options, executable_path=chromedriver_path)


### Exibindo Informações dos Markers